# Speakato sandbox - free model testing

In [1]:
import spacy
import pandas as pd
import numpy as np
import json
import keras.models

## Loading dataset and models

In [2]:
model_path = "../models/Speakato_model_2021-12-19"
dataset_path = "../examples/polish_commands_dataset"

with open(f"{model_path}/info.json", "r") as f:
    config = json.load(f)

with open(f"{dataset_path}/commands.txt", "r") as f:
    labels = [x.rstrip("\n") for x in f.readlines()]

config

{'language': 'pl',
 'lemmatizer': 'pl_core_news_sm',
 'token_len': 96,
 'train_evaluation': [0.0025778175331652164, 1.0],
 'test_evaluation': [1.948368787765503, 0.20000000298023224]}

In [3]:
nlp = spacy.load(config["lemmatizer"])
data = pd.read_json(f"{dataset_path}/dataset.json")
data

,text,command
0,Dzień dobry,greeting
1,Cześć,greeting
2,Hej,greeting
3,Siema,greeting
4,"Elo, co tam",greeting
5,Otwórz aplikację spotify,open
6,Włącz chrome,open
7,Otwórz notatnik,open
8,Włącz mi aplikacje notatnik,open
9,Uruchom przeglądarkę,open


In [4]:
model = keras.models.load_model(f"{model_path}/model")

## Evaluation and comparison of the results

In [5]:
X_raw = data["text"]
X = np.array([nlp(x).vector for x in X_raw])
y = pd.get_dummies(data[["command"]]).values

model.evaluate(X, y)

1/1 [==============================] - 1s 697ms/step - loss: 0.6106 - accuracy: 0.7500


[0.6106374859809875, 0.75]

In [6]:
y_predicted = model(X)

In [7]:
for i, x in enumerate(X_raw):
    correct_index = y[i].tolist().index(max(y[i]))
    recognized_index = y_predicted[i].numpy().tolist().index(max(y_predicted[i]))
    print(f"{x}\nRecognized: {labels[recognized_index]}\nCorrect: {labels[correct_index]}\n")

Dzień dobry
Recognized: close
Correct: close

Cześć
Recognized: close
Correct: close

Hej
Recognized: close
Correct: close

Siema
Recognized: close
Correct: close

Elo, co tam
Recognized: close
Correct: close

Otwórz aplikację spotify
Recognized: open
Correct: alarm

Włącz chrome
Recognized: alarm
Correct: alarm

Otwórz notatnik
Recognized: open
Correct: alarm

Włącz mi aplikacje notatnik
Recognized: alarm
Correct: alarm

Uruchom przeglądarkę
Recognized: alarm
Correct: alarm

Zamknij okno
Recognized: open
Correct: open

Zamknij aplikację
Recognized: open
Correct: open

Wyłącz aplikację
Recognized: open
Correct: open

Nastaw budzik na ośmą rano
Recognized: close
Correct: greeting

Obudź mnie o czwartej rano
Recognized: close
Correct: greeting

Włącz alarm na siedemnastą
Recognized: greeting
Correct: greeting

